In [ ]:
from pyomo.environ import *

model = ConcreteModel()

model.VEGETABLES = Set(initialize=['H1', 'H2', 'H3'])
model.X = RangeSet(0, 5)
model.TIME = RangeSet(0, 10)

model.growth = Param(model.VEGETABLES, initialize={'H1': 2, 'H2': 4, 'H3': 5})

model.scheduling = Var(model.X, model.VEGETABLES, model.TIME, domain=Binary)

def objective_rule(model):
    return sum(model.scheduling[x, h, t] / model.growth[h]
               for x in model.X for h in model.VEGETABLES for t in model.TIME)

model.objective = Objective(rule=objective_rule, sense=maximize)

def supply_unique_vegetables(model, h, t):
    return sum(model.scheduling[x, h, t] for x in model.X) <= 1

model.demand_constraint = Constraint(model.VEGETABLES, model.TIME, rule=supply_unique_vegetables)

solver = SolverFactory('glpk')
solver.options['tmlim'] = 60
results = solver.solve(model)

if results.solver.termination_condition == TerminationCondition.optimal:
    print("Optimal solution found")
    for x in model.X:
        for h in model.VEGETABLES:
            for t in model.TIME:
                print(f"Quantity from {x} to {h} at time {t}: {model.scheduling[x, h, t].value}")
    print("Total objective value:", model.objective())
else:
    print("Solver did not converge to an optimal solution")


Optimal solution found
Quantity from 0 to H1 at time 0: 1.0
Quantity from 0 to H1 at time 1: 1.0
Quantity from 0 to H1 at time 2: 1.0
Quantity from 0 to H1 at time 3: 1.0
Quantity from 0 to H1 at time 4: 1.0
Quantity from 0 to H1 at time 5: 1.0
Quantity from 0 to H1 at time 6: 1.0
Quantity from 0 to H1 at time 7: 1.0
Quantity from 0 to H1 at time 8: 1.0
Quantity from 0 to H1 at time 9: 1.0
Quantity from 0 to H1 at time 10: 1.0
Quantity from 0 to H2 at time 0: 1.0
Quantity from 0 to H2 at time 1: 1.0
Quantity from 0 to H2 at time 2: 1.0
Quantity from 0 to H2 at time 3: 1.0
Quantity from 0 to H2 at time 4: 1.0
Quantity from 0 to H2 at time 5: 1.0
Quantity from 0 to H2 at time 6: 1.0
Quantity from 0 to H2 at time 7: 1.0
Quantity from 0 to H2 at time 8: 1.0
Quantity from 0 to H2 at time 9: 1.0
Quantity from 0 to H2 at time 10: 1.0
Quantity from 0 to H3 at time 0: 1.0
Quantity from 0 to H3 at time 1: 1.0
Quantity from 0 to H3 at time 2: 1.0
Quantity from 0 to H3 at time 3: 1.0
Quantity from

In [3]:
from pyomo.environ import *

# Create an instance of a concrete model
model = ConcreteModel()

# Sets
model.VEGETABLES = Set(initialize=['H1', 'H2', 'H3'])
model.X = RangeSet(0,5)
model.TIME = RangeSet(0,10)
# Parameters
# Transportation costs from factories to warehouses
model.growth = Param(model.VEGETABLES, initialize={
    'H1': 5,
    'H2': 3,
    'H3': 3
})

# Variables
model.scheduling = Var(model.X, model.VEGETABLES, model.TIME, initialize=0, domain=Binary)

# Constraints
def supply_time_growth(model, x, h, t):
    
    if t == 0:
        return Constraint.Skip
    
    prev_hours = min(t, model.growth[h])
    
    prev_sum = sum(model.scheduling[x, h, t - t_i] for t_i in range(prev_hours))
    
    right_side =  min(value(model.scheduling[x, h, t - t_i]) for t_i in range(prev_hours))  
    # right_side = value(min(model.min_variable[x, h, t]))
    
    return  prev_sum * right_side  == prev_hours * right_side

def supply_unique_vegetables(model, x, t):
    return sum(model.scheduling[x, h, t] for h in model.VEGETABLES) <= 1  # Constraint on warehouses' demand

model.supply_constraint = Constraint(model.X, model.VEGETABLES, model.TIME, rule=supply_time_growth)
model.demand_constraint = Constraint(model.X, model.TIME, rule=supply_unique_vegetables)

# Objective function
def production(model):
    return sum(model.scheduling[x, h, t]/ model.growth[h]
               for x in model.X for h in model.VEGETABLES for t in model.TIME)

def demmand(model):
    return (5 - sum(model.scheduling[x, 'H1', t] for x in model.X for t in model.TIME)/ model.growth['H1']) +\
            (6 - sum(model.scheduling[x, 'H2', t]  for x in model.X for t in model.TIME)/ model.growth['H2']) +\
            (8 - sum(model.scheduling[x, 'H3', t] for x in model.X for t in model.TIME)/ model.growth['H3'] ) 

# model.production = Objective(rule=production, sense=maximize)
model.demmand = Objective(rule=demmand, sense=minimize)

# model.objective = Multi

# Solve the model
solver = SolverFactory('cbc')  # Using GLPK solver
results = solver.solve(model)

# # Display results
if results.solver.termination_condition == TerminationCondition.optimal:
    print("Optimal solution found")
    # for x in model.X:
    #     for h in model.VEGETABLES:
    #         for t in model.TIME:
    #             print(f"Quantity transported from {x} to {h} {t}: {model.scheduling[x, h, t].value}")
    print("Total cost:", model.demmand())
# else:
#     print("Solver did not converge to an optimal solution")


Optimal solution found
Total cost: -3.0


In [2]:
import sys
sys.path.append('/home/marnugue/Documents/crop_scheduling_optimization')

In [3]:
from src.utils.epsilon_constraint import EpsilonConstraintAugment

In [5]:
l = eps.execute()

TypeError: EpsilonConstraintAugment.__iterate_payoff_table() takes 2 positional arguments but 3 were given

In [40]:
results.solver.termination_condition

<TerminationCondition.optimal: 'optimal'>

In [39]:
for x in model.X:
    for h in model.VEGETABLES:
        for t in model.TIME:
            print(f"Quantity transported from {x} to {h} {t}: {model.scheduling[x, h, t].value}")

Quantity transported from 0 to H1 0: 1.0
Quantity transported from 0 to H1 1: 1.0
Quantity transported from 0 to H1 2: 1.0
Quantity transported from 0 to H1 3: 1.0
Quantity transported from 0 to H1 4: 1.0
Quantity transported from 0 to H1 5: 1.0
Quantity transported from 0 to H1 6: 1.0
Quantity transported from 0 to H1 7: 1.0
Quantity transported from 0 to H1 8: 1.0
Quantity transported from 0 to H1 9: 1.0
Quantity transported from 0 to H1 10: 1.0
Quantity transported from 0 to H2 0: 0.0
Quantity transported from 0 to H2 1: 0.0
Quantity transported from 0 to H2 2: 0.0
Quantity transported from 0 to H2 3: 0.0
Quantity transported from 0 to H2 4: 0.0
Quantity transported from 0 to H2 5: 0.0
Quantity transported from 0 to H2 6: 0.0
Quantity transported from 0 to H2 7: 0.0
Quantity transported from 0 to H2 8: 0.0
Quantity transported from 0 to H2 9: 0.0
Quantity transported from 0 to H2 10: 0.0
Quantity transported from 0 to H3 0: 0.0
Quantity transported from 0 to H3 1: 0.0
Quantity trans

In [15]:
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Sets, Parameters, Decision Variables, Constraints defined as before...

# Objective Functions
def cost_rule(model):
    return sum(model.Cost[p] * model.Quantity[p] for p in model.Products)
model.CostObjective = Objective(rule=cost_rule, sense=minimize)

def efficiency_rule(model):
    return sum(model.Efficiency[p] * model.Quantity[p] for p in model.Products)
model.EfficiencyObjective = Objective(rule=efficiency_rule, sense=maximize)

# Solve the multi-objective optimization problem
opt = SolverFactory('cbc')
opt.solve(model, strategy='nondominated')

# Display Results
print("Optimal Quantities:")
for p in model.Products:
    print(f"{p}: {model.Quantity[p].value}")

print("Total Cost:", model.CostObjective())
print("Total Efficiency:", model.EfficiencyObjective())


ERROR: Rule failed when generating expression for Objective CostObjective with
index None: AttributeError: 'ConcreteModel' object has no attribute 'Products'
ERROR: Constructing component 'CostObjective' from data=None failed:
AttributeError: 'ConcreteModel' object has no attribute 'Products'


AttributeError: 'ConcreteModel' object has no attribute 'Products'

# Epsilon implementation/

In [ ]:
from pyomo.environ import *
import matplotlib.pyplot as plt


# max f1 = X1 <br>
# max f2 = 3 X1 + 4 X2 <br>
# st  X1 <= 20 <br>
#     X2 <= 40 <br>
#     5 X1 + 4 X2 <= 200 <br>

model = ConcreteModel()

model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

model.C1 = Constraint(expr = model.X1 <= 20)
model.C2 = Constraint(expr = model.X2 <= 40)
model.C3 = Constraint(expr = 5 * model.X1 + 4 * model.X2 <= 200)

model.f1 = Var()
model.f2 = Var()
model.C_f1 = Constraint(expr= model.f1 == model.X1)
model.C_f2 = Constraint(expr= model.f2 == 3 * model.X1 + 4 * model.X2)
model.O_f1 = Objective(expr= model.f1  , sense=maximize)
model.O_f2 = Objective(expr= model.f2  , sense=maximize)

model.O_f2.deactivate()

solver = SolverFactory('cplex')
solver.solve(model);

print( '( X1 , X2 ) = ( ' + str(value(model.X1)) + ' , ' + str(value(model.X2)) + ' )')
print( 'f1 = ' + str(value(model.f1)) )
print( 'f2 = ' + str(value(model.f2)) )
f2_min = value(model.f2)


# ## max f2

model.O_f2.activate()
model.O_f1.deactivate()

solver = SolverFactory('cplex')
solver.solve(model);

print( '( X1 , X2 ) = ( ' + str(value(model.X1)) + ' , ' + str(value(model.X2)) + ' )')
print( 'f1 = ' + str(value(model.f1)) )
print( 'f2 = ' + str(value(model.f2)) )
f2_max = value(model.f2)


# ## apply normal $\epsilon$-Constraint

model.O_f1.activate()
model.O_f2.deactivate()

model.e = Param(initialize=0, mutable=True)

model.C_epsilon = Constraint(expr = model.f2 == model.e)

solver.solve(model);

print('Each iteration will keep f2 lower than some values between f2_min and f2_max, so ['       + str(f2_min) + ', ' + str(f2_max) + ']')

n = 4
step = int((f2_max - f2_min) / n)
steps = list(range(int(f2_min),int(f2_max),step)) + [f2_max]

x1_l = []
x2_l = []
for i in steps:
    model.e = i
    solver.solve(model);
    x1_l.append(value(model.X1))
    x2_l.append(value(model.X2))
plt.plot(x1_l,x2_l,'o-.');
plt.title('inefficient Pareto-front');
plt.grid(True);


# ## apply augmented $\epsilon$-Constraint

# max   f2 + delta*epsilon <br>
#  s.t. f2 - s = e

model.del_component(model.O_f1)
model.del_component(model.O_f2)
model.del_component(model.C_epsilon)

model.delta = Param(initialize=0.00001)

model.s = Var(within=NonNegativeReals)

model.O_f1 = Objective(expr = model.f1 + model.delta * model.s, sense=maximize)

model.C_e = Constraint(expr = model.f2 - model.s == model.e)

x1_l = []
x2_l = []
for i in range(160,190,6):
    model.e = i
    solver.solve(model);
    x1_l.append(value(model.X1))
    x2_l.append(value(model.X2))
plt.plot(x1_l,x2_l,'o-.');
plt.title('efficient Pareto-front');
plt.grid(True);